In [1]:
!conda install -c conda-forge gdcm -y

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdcm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.5                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    gdcm-2.8.9                 |   py37h71b2a6d_0         3.4 MB  conda-forge
    libjpeg-turbo-2.0.3        |       h516909a_1        1022 KB  conda-forge
    -------------------------------------

In [2]:
import gc
import pydicom
import scipy.ndimage
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from os import listdir, mkdir
from PIL import Image
from tqdm import tqdm
from datetime import datetime
from scipy.stats import describe
from scipy.ndimage import binary_fill_holes
from sklearn.cluster import KMeans, MiniBatchKMeans
from skimage import measure, morphology
from skimage.filters import threshold_otsu, median
from skimage.segmentation import clear_border

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
def get_slice_thickness(s0, s1):
    if "ImagePositionPatient" in s0.dir():
        return np.abs(s0.ImagePositionPatient[2] - s1.ImagePositionPatient[2])
    if "SliceLocation" in s0.dir():
        return np.abs(s0.SliceLocation - s1.SliceLocation)
    return s0.SliceThickness

def load_scans(dcm_path):
    files     = listdir(dcm_path)
    file_nums = [np.int(file.split(".")[0]) for file in files]
    sorted_file_nums = np.sort(file_nums)[::-1]
    slices = [pydicom.dcmread(dcm_path + "/" + str(file_num) + ".dcm" ) for file_num in sorted_file_nums]
    st     = get_slice_thickness(slices[0],slices[1])
    images = np.stack([file.pixel_array for file in slices])
    images = images.astype(np.int16)
    return slices, images, st, slices[0].PixelSpacing

def transform_to_hu(slices):
    images = np.stack([file.pixel_array for file in slices])
    images = images.astype(np.int16)

    # convert ouside pixel-values to air:
    # I'm using <= -1000 to be sure that other defaults are captured as well
    images[images <= -1000] = 0
    
    # convert to HU
    for n in range(len(slices)):
        intercept = slices[n].RescaleIntercept
        slope     = slices[n].RescaleSlope
        if slope != 1:
            images[n] = slope * images[n].astype(np.float64)
            images[n] = images[n].astype(np.int16)
        images[n] += np.int16(intercept)
    
    return np.array(images, dtype=np.int16)

In [4]:
def lung_segment_type1(img):
    if len(np.unique(img))==1:
        lungs  = np.zeros_like(img)
    else:
        thresh = threshold_otsu(img)
        binary = img <= thresh
        lungs  = median(clear_border(binary))
        lungs  = morphology.binary_closing(lungs, selem=morphology.disk(7))
        lungs  = binary_fill_holes(lungs)

    final = lungs*img
    final[final == 0] = np.min(img)

    return final, lungs

def lung_segment_type2(img):
    binary_image = np.array(img > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)

    background_label_1 = labels[0,0]
    background_label_2 = labels[0,-1]
    background_label_3 = labels[-1,0]
    background_label_4 = labels[-1,-1]

    #Fill the air around the person
    binary_image[background_label_1 == labels] = 2
    binary_image[background_label_2 == labels] = 2
    binary_image[background_label_3 == labels] = 2
    binary_image[background_label_4 == labels] = 2

    #We have a lot of remaining small signals outside of the lungs that need to be removed. 
    #In our competition closing is superior to fill_lungs 
    binary_image = morphology.closing(binary_image, selem=morphology.disk(4))

    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1

    return binary_image*img, binary_image

def lung_segment_type3(img):
    img_org = img.copy()
    row_size, col_size = img.shape
    img = (img-np.mean(img))/np.std(img)
    # Find the average pixel value near the lungs to renormalize washed out images
    middle = img[int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)] 
    mean   = np.mean(middle)  
    max    = np.max(img)
    min    = np.min(img)
    
    # To improve threshold finding, I'm moving the underflow and overflow on the pixel spectrum
    img[img==max]=mean
    img[img==min]=mean
    
    # Using Kmeans to separate foreground (soft tissue / bone) and background (lung/air)
    kmeans  = MiniBatchKMeans(n_clusters=2,batch_size=3000).fit(middle.reshape(-1,1))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold  = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image

    # First erode away the finer elements, then dilate to include some of the pixels surrounding the lung.  
    # We don't want to accidentally clip the lung.
    eroded   = morphology.erosion(thresh_img,np.ones([3,3]))
    dilation = morphology.dilation(eroded,np.ones([8,8]))

    labels      = measure.label(dilation) # Different labels are displayed in different colors
    label_vals  = np.unique(labels)
    regions     = measure.regionprops(labels)

    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<row_size/10*9 and B[3]-B[1]<col_size/10*9 and B[0]>row_size/5 and B[2]<col_size/5*4:
            good_labels.append(prop.label)
    mask    = np.ndarray([row_size,col_size],dtype=np.int8)
    mask[:] = 0

    # After just the lungs are left, we do another large dilation in order to fill in and out the lung mask 
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation
    
    return img_org*mask, mask

def lung_segment_stack(imgs, method="type1"):
    masks    = np.empty_like(imgs)
    segments = np.empty_like(imgs)

    for i, img in enumerate(imgs):
        if   method=="type1":
            seg, mask = lung_segment_type1(img)
        elif method=="type2":
            seg, mask = lung_segment_type2(img)
        elif method=="type3":
            seg, mask = lung_segment_type3(img)
        segments[i,:,:] = seg
        masks[i,:,:]    = mask
        
    return segments, masks

def lung_volume(st, ps, masks):
    return np.round(np.sum(masks) * st * ps[0] * ps[1], 3)

def hist_analysis(segmented):
    values = segmented.flatten()
    values = values[values >= -1000]
    stats  = describe(values)
    return stats.mean, stats.variance, stats.skewness, stats.kurtosis

def chest_measurements(ps, masks):
    middle_slice = masks[len(masks)//2]
    lung_area    = np.round(np.sum(middle_slice.flatten()) * ps[0] * ps[1], 3)
    conv_h = morphology.convex_hull_image(middle_slice)
    props  = measure.regionprops(measure.label(conv_h))
    chest_diameter = np.round(props[0].major_axis_length, 3)
    chest_circ     = np.round(props[0].perimeter, 3)
    return lung_area, chest_diameter, chest_circ

In [5]:
def resize_scan(scan, new_shape):
    # read slice as 32 bit signed integers
    img = Image.fromarray(scan, mode="I")
    # do the resizing
    img = img.resize(new_shape, resample=Image.LANCZOS)
    # convert back to 16 bit integers
    resized_scan = np.array(img, dtype=np.int16)
    return resized_scan

def crop_scan(scan):
    img = Image.fromarray(scan, mode="I")
    
    left = (scan.shape[0]-512)/2
    right = (scan.shape[0]+512)/2
    top = (scan.shape[1]-512)/2
    bottom = (scan.shape[1]+512)/2

    img = img.crop((left, top, right, bottom))
    # convert back to 16 bit integers
    cropped_scan = np.array(img, dtype=np.int16)
    return cropped_scan

def crop_and_resize(scan, new_shape):
    img = Image.fromarray(scan, mode="I")
    
    left = (scan.shape[0]-512)/2
    right = (scan.shape[0]+512)/2
    top = (scan.shape[1]-512)/2
    bottom = (scan.shape[1]+512)/2
    
    img = img.crop((left, top, right, bottom))
    img = img.resize(new_shape, resample=Image.LANCZOS)
    
    cropped_resized_scan = np.array(img, dtype=np.int16)
    return cropped_resized_scan

In [6]:
def preprocess_to_hu_scans(path, my_shape, output_dir):
    
    for i, patient in enumerate(tqdm(listdir(path))):
        scans    = load_scans(path + "/" + patient + "/")
        hu_scans = transform_to_hu(scans) 
        prepared_scans = np.zeros((hu_scans.shape[0], my_shape[0], my_shape[1]),
                                  dtype=np.int16)
        # if squared:
        if hu_scans.shape[1] == hu_scans.shape[2]:
            
            # if size is as desired
            if hu_scans.shape[1] == my_shape[0]:
                continue
            # else resize:
            else:
               # as we have not converted to jpeg to keep all information, we need to do a workaround
                hu_scans = hu_scans.astype(np.int32)
                for s in range(hu_scans.shape[0]): 
                    prepared_scans[s] = resize_scan(hu_scans[s,:,:], my_shape)

        # if non-squared - do a center crop to 512, 512 and then resize to desired shape
        else:
            hu_scans = hu_scans.astype(np.int32)
            for s in range(hu_scans.shape[0]):
                # if desired shape is 512x512:
                if my_shape[0]==512:
                    prepared_scans[s] = crop_scan(hu_scans[s,:,:])
                else:
                    prepared_scans[s] = crop_and_resize(hu_scans[s,:,:], my_shape)
                
        # save the prepared scans of patient:
        np.save(output_dir + "/" + patient + '_hu_scans', prepared_scans)

In [7]:
def resize_image(images, resized_shape):
    images         = images.astype(np.int32)
    resized_images = np.zeros((images.shape[0], resized_shape[0], resized_shape[1]),
                              dtype=np.int16)
    # if squared:
    if images.shape[1] == images.shape[2]:
        # if size is not as desired
        if images.shape[1] == resized_shape[0]:
            return images
        # else resize:
        else:
            for s in range(images.shape[0]):
                resized_images[s] = resize_scan(images[s,:,:], resized_shape)

    # if non-squared - do a center crop to 512, 512 and then resize to desired shape
    else:
        for s in range(images.shape[0]):
            # if desired shape is 512x512:
            if resized_shape[0]==512:
                resized_images[s] = crop_scan(images[s,:,:])
            else:
                resized_images[s] = crop_and_resize(images[s,:,:], resized_shape)
    return resized_images

In [8]:
def calculate_lung_area(path, method="type1", resize_shape=(512,512)):
    dicom_data = []
    for i, patient in enumerate(tqdm(listdir(path))):
        scans, images, st, ps = load_scans(path + "/" + patient + "/")
        if method in ("type1","type2"):
            images = transform_to_hu(scans)

        images = resize_image(images, resize_shape)
        segmented, masks = lung_segment_stack(images, method)
        del scans, images
        gc.collect()

        vol = lung_volume(st, ps, masks)
        m,v,s,k = hist_analysis(segmented)
        lung_area, chest_diameter, chest_circ = chest_measurements(ps, masks)
        del segmented, masks
        gc.collect()

        dicom_data.append([patient, vol, m, v, s, k, 
                           lung_area, chest_diameter, chest_circ])
        
    cols = ["patient","volume","mean","variance","skewness","kurtosis",
            "lung_area","chest_diameter","chest_circ"]
    return pd.DataFrame(dicom_data, columns=cols)

In [9]:
basepath = "../input/osic-pulmonary-fibrosis-progression/train"

In [10]:
%%time
df = calculate_lung_area(basepath, "type1", (256,256))

100%|██████████| 176/176 [26:24<00:00,  9.00s/it]

CPU times: user 23min 34s, sys: 1min 28s, total: 25min 2s
Wall time: 26min 24s


In [11]:
df.head()

,patient,volume,mean,variance,skewness,kurtosis,lung_area,chest_diameter,chest_circ
0,ID00123637202217151272140,567435.130,-637.928942,58488.331748,1.307111,1.252924,5580.778,208.639,625.889
1,ID00407637202308788732304,828860.474,-735.302917,54572.450836,1.985335,3.937870,5505.371,166.293,471.463
2,ID00023637202179104603099,659613.819,-739.666131,51671.589128,1.856592,3.422155,5027.508,212.359,674.458
3,ID00267637202270790561585,813490.234,-625.407140,50376.678410,1.165050,1.569063,3555.469,209.455,634.617
4,ID00224637202259281193413,227540.329,-730.186856,55789.200738,2.056116,4.102675,7663.482,191.707,554.818


In [12]:
df.to_csv("./dicom_data.csv", index=False)

In [13]:
generate   = False
output_dir = "scans_512x512"
my_shape   = (512, 512)
if generate:
    mkdir(output_dir)
    preprocess_to_hu_scans(basepath, my_shape, output_dir)